In [2]:
!pip install datasets
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [7]:
dataset_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather", 
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-21',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-31',
)

print('Loading is done!')

Loading dataset with config: PatentsConfig(name='sample', version=0.0.0, data_dir='sample', data_files={'train': ['https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather']}, description='Patent data from January 2016, for debugging')


Using metadata file: /root/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710


Reading metadata file: /root/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710
Filtering train dataset by filing start date: 2016-01-01
Filtering train dataset by filing end date: 2016-01-21
Filtering val dataset by filing start date: 2016-01-22
Filtering val dataset by filing end date: 2016-01-31


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset hupd downloaded and prepared to /root/.cache/huggingface/datasets/HUPD___hupd/sample-b25623b556c03ec2/0.0.0/6920d2def8fd7767046c0470603357f76866e5a09c97e19571896bfdca521142. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Loading is done!


In [8]:
raw_data = load_dataset("glue","mrpc")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['patent_number', 'decision', 'title', 'abstract', 'claims', 'background', 'summary', 'description', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id'],
        num_rows: 16153
    })
    validation: Dataset({
        features: ['patent_number', 'decision', 'title', 'abstract', 'claims', 'background', 'summary', 'description', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id'],
        num_rows: 9094
    })
})


In [10]:
print(raw_data)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [11]:
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [12]:
def tokenize_function(dataset):
  return tokenizer(dataset["abstract"], dataset["claims"], truncation=True)

In [13]:
tokenized_ds = dataset_dict.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/16153 [00:00<?, ? examples/s]

Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
training_args = TrainingArguments("test-trainer")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

ValueError: ignored

In [1]:
from transformers import BertForSequenceClassification, BertTokenizer

model_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

C:\Users\dunga\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassificatio

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

dataset_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather", 
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-21',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-31',
)

print('Loading is done!')

Found cached dataset hupd (C:/Users/dunga/.cache/huggingface/datasets/HUPD___hupd/sample-b25623b556c03ec2/0.0.0/6920d2def8fd7767046c0470603357f76866e5a09c97e19571896bfdca521142)
100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.25it/s]

Loading is done!


In [4]:
def tokenize_data(data):
    return tokenizer(data['abstract'], truncation=True, padding=True)

tokenized_train = dataset_dict['train'].map(tokenize_data, batched=True)
tokenized_val = dataset_dict['validation'].map(tokenize_data, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

Loading cached processed dataset at C:\Users\dunga\.cache\huggingface\datasets\HUPD___hupd\sample-b25623b556c03ec2\0.0.0\6920d2def8fd7767046c0470603357f76866e5a09c97e19571896bfdca521142\cache-4c49c86a14694802.arrow
Loading cached processed dataset at C:\Users\dunga\.cache\huggingface\datasets\HUPD___hupd\sample-b25623b556c03ec2\0.0.0\6920d2def8fd7767046c0470603357f76866e5a09c97e19571896bfdca521142\cache-63b3029adc318c7c.arrow


In [5]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [6]:
trainer.train()

C:\Users\dunga\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [ ]:
trainer.evaluate()


In [ ]:
def predict(model, data):
    inputs = tokenizer(data['abstract'], truncation=True, padding=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    return {'patentability_score': predicted_class}

model.to('cuda')
result = predict(model, {'abstract': 'This is an abstract'})

In [ ]:
#############

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

dataset_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather", 
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-21',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-31',
)

C:\Users\dunga\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset hupd (C:/Users/dunga/.cache/huggingface/datasets/HUPD___hupd/sample-b25623b556c03ec2/0.0.0/6920d2def8fd7767046c0470603357f76866e5a09c97e19571896bfdca521142)
100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.52it/s]


In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def preprocess(example):
    if example['decision'] == 'ACCEPTED':
        label = 0
    elif example['decision'] == 'REJECTED':
        label = 1
    else:
        label = None

    if label is not None:
        encoded = tokenizer.encode_plus(example['abstract'],
                                         max_length=512,
                                         padding='max_length',
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
        return {'input_ids': encoded['input_ids'][0],
                'attention_mask': encoded['attention_mask'][0],
                'label': label}

train_dataset = dataset_dict['train'].filter(lambda example: example['decision'] in ['ACCEPTED', 'REJECTED']).map(preprocess)
val_dataset = dataset_dict['validation'].filter(lambda example: example['decision'] in ['ACCEPTED', 'REJECTED']).map(preprocess)

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=50,
    learning_rate=5e-5,
    save_total_limit=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Step,Training Loss,Validation Loss
50,0.548600,0.536803


In [1]:
##############

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

dataset_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather", 
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-21',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-31',
)

print('Loading is done!')

C:\Users\dunga\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset hupd (C:/Users/dunga/.cache/huggingface/datasets/HUPD___hupd/sample-b25623b556c03ec2/0.0.0/6920d2def8fd7767046c0470603357f76866e5a09c97e19571896bfdca521142)
100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.36it/s]

Loading is done!


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# set the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)



Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

In [5]:
def preprocess(example):
    if example['decision'] == 'ACCEPTED':
        label = 0
    elif example['decision'] == 'REJECTED':
        label = 1
    else:
        label = None

    if label is not None:
        encoded = tokenizer.encode_plus(example['abstract'],
                                         max_length=512,
                                         padding='max_length',
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
        return {'input_ids': encoded['input_ids'][0],
                'attention_mask': encoded['attention_mask'][0],
                'label': label}

train_dataset = dataset_dict['train'].filter(lambda example: example['decision'] in ['ACCEPTED', 'REJECTED']).map(preprocess)
val_dataset = dataset_dict['validation'].filter(lambda example: example['decision'] in ['ACCEPTED', 'REJECTED']).map(preprocess)

Loading cached processed dataset at C:\Users\dunga\.cache\huggingface\datasets\HUPD___hupd\sample-b25623b556c03ec2\0.0.0\6920d2def8fd7767046c0470603357f76866e5a09c97e19571896bfdca521142\cache-e5c65470884a0973.arrow
Loading cached processed dataset at C:\Users\dunga\.cache\huggingface\datasets\HUPD___hupd\sample-b25623b556c03ec2\0.0.0\6920d2def8fd7767046c0470603357f76866e5a09c97e19571896bfdca521142\cache-9353e17d454d29bd.arrow


In [10]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    eval_steps=500,   # set evaluation steps
    save_strategy='steps',   # explicitly set save strategy to steps
    save_steps=500,   # set save steps
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
     #                           'attention_mask': torch.stack([f[1] for f in data]),
     #                           'labels': torch.stack([f[2] for f in data])},
    tokenizer=tokenizer,
)


In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [ ]:
# replace 'model_path' with the path where you want to save the model
model_path = '/path/to/save/model'

# save the model
model.save_pretrained(model_path)